In [2]:
##### Step 1: import functions #####
from keras.layers import Dense, Flatten, BatchNormalization, Activation, Conv2D, Conv1D, AveragePooling2D, AveragePooling1D, Input, MaxPooling1D, Dropout
from keras.models import load_model, Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras import backend as K, initializers, regularizers
from keras.metrics import mean_absolute_error, mean_squared_error
import pickle
import pandas as pd
import numpy as np
# from numpy.random import seed; seed(473)
import random
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
# from tensorflow import set_random_seed; set_random_seed(763)
from keras.losses import binary_crossentropy
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
# from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz, DecisionTreeRegressor
from sklearn.calibration import calibration_curve
from sklearn.utils import class_weight
from scipy.stats import ttest_ind
from IPython.display import Image  
import pydotplus
import math

In [ ]:
##### Step 2: load data #####
x = np.load('data/d13_unfiltered_all_log_stdrd.npy')
metaData = pd.read_csv('data/d13_annotations.csv', delimiter=';')
y = metaData['D13 PDX/NKX'].values

# filter out samples with NaN values
nan_idx = np.isnan(y)
x = x[~nan_idx]
y = y[~nan_idx]
samples = metaData['sample'].values[~nan_idx]

print('Total number of samples: ', x.shape[0])

# test_folds = np.load('data/d13_test_folds.npy', allow_pickle=True)
# valid_folds = np.load('data/d13_valid_folds.npy', allow_pickle=True)

In [ ]:
# find amount of entries in y below 30.0, between 30.0 and 40.0 and above 40.0
below_30 = np.sum(y < 30.0)
between_30_40 = np.sum((y >= 30.0) & (y < 40.0))
above_40 = np.sum(y >= 40.0)
print('Below 30.0: ', below_30)
print('Between 30.0 and 40.0: ', between_30_40)
print('Above 40.0: ', above_40)

In [3]:
test_folds = np.load('data/d13_test_folds.npy', allow_pickle=True)
valid_folds = np.load('data/d13_valid_folds.npy', allow_pickle=True)

In [7]:
##### Step 3: define test and train sets #####
# shuffle data
idx = np.arange(x.shape[0])
np.random.shuffle(idx)
shuffled_samples = samples[idx]
test_folds = np.array_split(shuffled_samples, 7)

# create 5-fold train-validation splits for each test fold
valid_folds = []
for test_fold in test_folds:
    idx_train = np.isin(shuffled_samples, test_fold, invert=True)
    samples_train = shuffled_samples[idx_train]
    
    valid_fold = np.array_split(samples_train, 5)
    valid_folds.append(valid_fold)

In [4]:
##### Step 4: define model #####

def create_model(x, l2=0.001, dropout=1/3):
    # input
    model_input = Input(shape=x[0].shape)

    # first convolution layer
    model_output = Conv1D(3, 
                        kernel_size=1, 
                        kernel_initializer=initializers.RandomUniform(),
                        kernel_regularizer=regularizers.l2(l2),
                        activation=None)(model_input)
    model_output = BatchNormalization()(model_output)
    model_output = Activation("relu")(model_output)

    # # second convolution layer
    # model_output = Conv1D(4,
    #                     kernel_size=1,
    #                     kernel_initializer=initializers.RandomUniform(),
    #                     kernel_regularizer=regularizers.l2(l2),
    #                     activation=None)(model_output)
    # model_output = BatchNormalization()(model_output)
    # model_output = Activation("relu")(model_output)

    # pooling layer
    model_output = AveragePooling1D(pool_size=x.shape[1])(model_output)
    model_output = Flatten()(model_output)

    # # dropout
    # model_output = Dropout(rate=1/4)(model_output)

    # Dense layer
    model_output = Dense(3, 
                        kernel_initializer=initializers.RandomUniform(),
                        kernel_regularizer=regularizers.l2(l2),
                        activation=None)(model_output)
    model_output = BatchNormalization()(model_output)
    model_output = Activation("relu")(model_output)

    # output layer
    model_output = Dense(1, 
                        kernel_initializer=initializers.RandomUniform(),
                        activation=None)(model_output)
    model_output = BatchNormalization()(model_output)
    model_output = Activation("linear")(model_output)

    return Model(inputs=model_input, outputs=model_output)

# define function for plotting train and validation loss and accuracy
def plot_loss(history):
    # increase font size
    plt.rcParams.update({'font.size': 14})

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('train vs validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='best')
    plt.show()

def calc_mean_std(history):
    mean = np.array(history).mean(axis=0)
    std = np.array(history).std(axis=0)
    return mean, std

def plot_error_bar(train_mean, train_std, val_mean, val_std, ylabel, ax, leg_loc='best'):
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.errorbar(range(1, len(train_mean) + 1), train_mean, yerr=train_std, label='train')
    ax.errorbar(range(1, len(val_mean) + 1), val_mean, yerr=val_std, label='validation')
    ax.set_xlabel('epoch')
    ax.set_ylabel(ylabel)
    # ax.set_xticks(range(1, len(train_mean) + 1))
    ax.set_ylim(bottom=0)
    ax.legend(loc=leg_loc)

def custom_mse(y_true, y_pred):
    return np.mean(np.square(y_pred - y_true), axis=-1)

In [ ]:
##### Step 5: train and test model #####

# implement a learning rate schedule which starts at 0.1, then decreases with a rate of 0.9 in 100 steps
def lr_schedule(epoch, lr):
    if epoch > 100:
        return lr*0.97
    else:
        return lr

for test_round, test_fold in enumerate(test_folds):
    print(f"Test fold {test_round+1}/{len(test_folds)}")
    
    idx_test = np.isin(samples, test_fold)
    idx_train = np.invert(idx_test)

    x_train = x[idx_train,:,:]; y_train = y[idx_train]; samples_train = samples[idx_train]
    x_test = x[idx_test,:,:]; y_test = y[idx_test]; samples_test = samples[idx_test]

    # Lists to store evaluation results
    val_loss_history = []; val_mae_history = []
    train_loss_history = []; train_mae_history = []

    mae_best_epoch = []; loss_best_epoch = []

    # for fold, (train_indices, valid_indices) in enumerate(kf.split(x_train)):
    for fold, valid_samples in enumerate(valid_folds[test_round]):
        print(f"Training fold {fold+1}/{len(valid_folds[test_round])}")

        valid_indices = np.isin(samples_train, valid_samples)
        train_indices = np.invert(valid_indices)
        
        x_train_fold = x_train[train_indices]
        y_train_fold = y_train[train_indices]
        x_valid_fold = x_train[valid_indices]
        y_valid_fold = y_train[valid_indices]
        
        # Build the model (same as your previous code)
        model = create_model(x_train, l2=0.00)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.1))

        lr_scheduler = LearningRateScheduler(lr_schedule)
        checkpointer = ModelCheckpoint(filepath=f'saved_weights_d13_pred_testfold{test_round+1}_fold{fold+1}.hdf5', 
                                    monitor='val_loss', verbose=0, 
                                    save_best_only=True)
        
        # Train the model
        history = model.fit(x_train_fold, y_train_fold,
                            batch_size=5,
                            epochs=100, 
                            verbose=0,
                            callbacks=[checkpointer, lr_scheduler],
                            validation_data=([x_valid_fold], y_valid_fold)
                            )
        
        plot_loss(model.history)
        
        # Store validation metrics for each fold
        val_loss_history.append(history.history['val_loss'])
        train_loss_history.append(history.history['loss'])

        # Find the epoch with the lowest validation loss
        best_epoch = np.argmin(history.history['val_loss'])
        loss_best_epoch.append(history.history['val_loss'][best_epoch])

        # print validation accuracy and loss at best epoch
        print(f"Validation loss at best epoch: {history.history['val_loss'][best_epoch]:.2f}")
    
    # print model with lowest validation loss and highest validation accuracy
    print(f"Model with lowest validation loss at best epoch: {np.argmin(loss_best_epoch)+1}")

    # Calculate average validation loss and accuracy for each model
    avg_val_loss = np.array(val_loss_history).mean(axis=1)

    # Choose the model with the highest average validation accuracy
    print(f"Model with lowest average validation loss: {np.argmin(avg_val_loss)+1}")

    # plot average validation loss
    [train_mean, train_std] = calc_mean_std(train_loss_history)
    [val_mean, val_std] = calc_mean_std(val_loss_history)
    print(f"Average train loss: {train_mean[-1]:.2f} +/- {train_std[-1]:.2f}")
    print(f"Average validation loss: {val_mean[-1]:.2f} +/- {val_std[-1]:.2f}")

    fig, ax = plt.subplots(1,1,figsize=(8,6))
    plot_error_bar(train_mean, train_std, val_mean, val_std, 'loss', ax, leg_loc='upper right')
    plt.show()

    # test models on test fold
    y_5fold_pred = np.zeros((y_test.shape[0], 5))

    for i in range(5):
        final_model = load_model(f'saved_weights_d13_pred_testfold{test_round+1}_fold{i+1}.hdf5', compile=False)

        # define loss function and optimizer
        final_model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.1))

        # generate ROC and AUC
        y_scores = final_model.predict([x_test])
        y_5fold_pred[:,i] = y_scores.flatten()

    # print prediction and true label
    df = pd.DataFrame({'sample': samples[idx_test],
                        'model 1': y_5fold_pred[:,0],
                        'model 2': y_5fold_pred[:,1],
                        'model 3': y_5fold_pred[:,2],
                        'model 4': y_5fold_pred[:,3],
                        'model 5': y_5fold_pred[:,4],
                        'true': y_test})
    print(df)
    
    # compute mean absolute error and mean squared error
    mae = np.abs(df.iloc[:,1:-1] - df['true'].values.reshape(-1,1)).mean(axis=0)
    # round to 2 decimal places
    mae = np.round(mae, 2)
    print(f"Mean absolute error: \n{mae}")

    # compute total sum of squares
    tss = np.square(df['true'] - df['true'].mean()).sum()

    # compute residual sum of squares
    rss = np.square(df.iloc[:,1:-1] - df['true'].values.reshape(-1,1)).sum(axis=0)

    # compute R2
    r2 = 1 - rss/tss
    # round to 2 decimal places
    r2 = np.round(r2, 2)
    print(f"R2: \n{r2}")

    # create dataframe for mae, mse and r2
    df_metrics = pd.DataFrame({'model': ['model 1', 'model 2', 'model 3', 'model 4', 'model 5'],
                        'mae': mae,
                        'r2': r2})
    
    plt.rcParams['figure.figsize'] = [9, 6]
    plt.rcParams.update({'font.size': 16})

    plt.plot(df.index, df['true'].values, 'ys', color='black', linestyle='-')
    plt.plot(df.index, df['model 1'].values, 'x', color='red', label='$MAE = $' + str(mae[0]), linestyle='-')
    plt.plot(df.index, df['model 2'].values, 'x', color='green', label='$MAE = $' + str(mae[1]), linestyle='-')
    plt.plot(df.index, df['model 3'].values, 'x', color='blue', label='$MAE = $' + str(mae[2]), linestyle='-')
    plt.plot(df.index, df['model 4'].values, 'x', color='purple', label='$MAE = $' + str(mae[3]), linestyle='-')
    plt.plot(df.index, df['model 5'].values, 'x', color='darkorange', label='$MAE = $' + str(mae[4]), linestyle='-')
    plt.ylim(bottom=0, top=100)
    plt.ylabel('PDX1/NKX6.1 double positives (%)')
    plt.xticks(df.index, df['sample'].values, rotation=45)
    plt.legend()
    plt.savefig(f'd13_pred_testfold{test_round+1}.png')
    plt.show()

    colors = ['red', 'green', 'blue', 'purple', 'darkorange']
    for i in range(5):
        plt.rcParams['figure.figsize'] = [9, 6]
        # Plot predictions
        column = 'model ' + str(i+1)
        plt.plot(df.index, df['true'].values, 'ys', color='black', linestyle='-', label='true values')
        plt.plot(df.index, df[column].values, 'x', color=colors[i], label='predicted values', linestyle='-')
        plt.fill_between(df.index, df['true'], df[column], color=colors[i], alpha=0.3)
        plt.bar(df.index, np.abs(df[column] - df['true']), color=colors[i], alpha=0.3, label='absolute residuals')
        plt.plot(df.index, mae[i]*np.ones(df.shape[0]), color=colors[i], linestyle='--', label='mean absolute error')
        plt.text(df.index[0], mae[i] + 2, str(mae[i]), fontsize=16, ha='center')
        plt.ylim(bottom=0, top=100)
        plt.ylabel('PDX1/NKX6.1 double positives (%)')
        plt.xticks(df.index, df['sample'].values, rotation=45)
        plt.yticks(np.arange(0, 101, 10))
        plt.legend(fontsize=14)
        plt.grid(True)
        plt.savefig(f'd13_pred_testfold{test_round+1}_model{i+1}_mae.png')
        plt.show()

        plt.scatter(df['true'], df[column], color=colors[i], label='$MAE = $' + str(mae[i]))
        plt.plot([0, 100], [0, 100], 'k--')
        plt.xlabel('true')
        plt.ylabel('predicted')
        ax = plt.gca()
        ax.set_aspect('equal', adjustable='box')
        plt.legend(fontsize=14)
        plt.savefig(f'd13_pred_testfold{test_round+1}_model{i+1}_scatter.png')
        plt.show()
    
    # save predictions
    df.to_csv(f'd13_pred_testfold{test_round+1}.csv', sep=';')
    df_metrics.to_csv(f'd13_metrics_testfold{test_round+1}.csv', sep=';')